In [3]:
import pandas as pd
import numpy as np
import time
import io
import requests
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection, svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

#Get Processed Data
url="https://raw.githubusercontent.com/mgcresswell/TCSS555-Project/main/deceptive-opinion_processed.csv"
s=requests.get(url).content
Corpus = pd.read_csv(io.StringIO(s.decode('utf-8')))

y = Corpus['deceptive']
X = Corpus.drop(['id','deceptive'], axis=1)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
hotelEncoded = label_encoder.fit_transform(X['hotel'])
polarityEncoded = label_encoder.fit_transform(X['polarity'])
sourceEncoded = label_encoder.fit_transform(X['source'])

onehot_encoder = OneHotEncoder(sparse=False)
hotelEncoded = hotelEncoded.reshape(len(hotelEncoded), 1)
X['hotel'] = onehot_encoder.fit_transform(hotelEncoded)
polarityEncoded = polarityEncoded.reshape(len(polarityEncoded), 1)
X['polarity'] = onehot_encoder.fit_transform(polarityEncoded)
sourceEncoded = sourceEncoded.reshape(len(sourceEncoded), 1)
X['source'] = onehot_encoder.fit_transform(sourceEncoded)

Tfidf_vect = TfidfVectorizer(max_features=3500)
Tfidf_vect.fit(Corpus['text'])
X_Tfidf = Tfidf_vect.transform(X['text'])
X['text'] = X_Tfidf.toarray()

split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
for train_index, test_index in split.split(X, y):
   Train_X, Test_X = X.loc[train_index], X.loc[test_index]
   Train_Y, Test_Y = y[train_index], y[test_index]

model = LogisticRegression(C=1.0).fit(Train_X, Train_Y)
start = time.perf_counter()
model.fit(Train_X, Train_Y)
stop = time.perf_counter()

#Test Accuracy 
y_pred = model.predict(Test_X)
test_Accuracy = accuracy_score(Test_Y, y_pred)*100

#Test Accuracy 
y_pred = model.predict(Train_X)
train_Accuracy = accuracy_score(Train_Y, y_pred)*100

crossvalMean = cross_val_score(model, X, y, cv=10).mean()

curTime = stop - start;
print(f"Training Time = {curTime:0.8f} Seconds")
print(f"Average Accuracy = {test_Accuracy}")
print(f"Average Training Time = {train_Accuracy}")
print(f"Cross Validation Mean = {crossvalMean}")



Training Time = 0.00554361 Seconds
Average Accuracy = 100.0
Average Training Time = 100.0
Cross Validation Mean = 1.0


In [6]:
import pandas as pd
import numpy as np
import time
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection, svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

url="https://raw.githubusercontent.com/mgcresswell/TCSS555-Project/main/deceptive-opinion_processed.csv"
s=requests.get(url).content
Corpus = pd.read_csv(io.StringIO(s.decode('utf-8'))) 

url="https://raw.githubusercontent.com/mgcresswell/TCSS555-Project/main/deceptive-opinion.csv"
s=requests.get(url).content
raw = pd.read_csv(io.StringIO(s.decode('utf-8'))) 

y = Corpus['deceptive']
X = Corpus.drop(['id','deceptive','source'], axis=1)

#feature engineering
punc = ['`','~','!','(',')','_','-','{','[','}','}',':',';','"',',','.','?','/','""']
X['char_count'] = raw["text"].apply(lambda x: sum(len(word) for word in str(x).split(" ")))
X['total_length'] = raw['text'].apply(len)
X['punc_count'] = raw['text'].apply(lambda x : len([a for a in x if a in punc]))
X['word_count'] = raw["text"].apply(lambda x: len(str(x).split(" ")))
X['char_count'] = raw["text"].apply(lambda x: sum(len(word) for word in str(x).split(" ")))
X['sentence_count'] = raw["text"].apply(lambda x: len(str(x).split(".")))
X['avg_word_length'] = X['char_count'] / X['word_count']
X['avg_sentence_length'] = X['word_count'] / X['sentence_count']
X['word_density'] = X['word_count'] / (X['char_count'] + 1)
X['punc_count'] = raw['text'].apply(lambda x : len([a for a in x if a in punc]))
X['total_length'] = raw['text'].apply(len)
X['capitals'] = raw['text'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
X['num_exclamation_marks'] = raw['text'].apply(lambda x: x.count('!'))
X['num_question_marks'] = raw['text'].apply(lambda x: x.count('?'))
X['num_punctuation'] = raw['text'].apply(lambda x: sum(x.count(w) for w in '.,;:'))
X['num_symbols'] = raw['text'].apply(lambda x: sum(x.count(w) for w in '*&$%'))
X['num_unique_words'] = raw['text'].apply(lambda x: len(set(w for w in x.split())))
X['words_vs_unique'] = X['num_unique_words'] / X['word_count']
X["word_unique_percent"] =  X["num_unique_words"]*100/X['word_count']

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
hotelEncoded = label_encoder.fit_transform(X['hotel'])
polarityEncoded = label_encoder.fit_transform(X['polarity'])
#sourceEncoded = label_encoder.fit_transform(X['source'])

onehot_encoder = OneHotEncoder(sparse=False)
hotelEncoded = hotelEncoded.reshape(len(hotelEncoded), 1)
X['hotel'] = onehot_encoder.fit_transform(hotelEncoded)
polarityEncoded = polarityEncoded.reshape(len(polarityEncoded), 1)
X['polarity'] = onehot_encoder.fit_transform(polarityEncoded)
#sourceEncoded = sourceEncoded.reshape(len(sourceEncoded), 1)
#X['source'] = onehot_encoder.fit_transform(sourceEncoded)

Tfidf_vect = TfidfVectorizer(max_features=3500)
Tfidf_vect.fit(Corpus['text'])
X_Tfidf = Tfidf_vect.transform(X['text'])
X['text'] = X_Tfidf.toarray()

split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
for train_index, test_index in split.split(X, y):
   Train_X, Test_X = X.loc[train_index], X.loc[test_index]
   Train_Y, Test_Y = y[train_index], y[test_index]

model = LogisticRegression(C=1.0, max_iter=10000).fit(Train_X, Train_Y)
start = time.perf_counter()
model.fit(Train_X, Train_Y)
stop = time.perf_counter()

#Test Accuracy 
y_pred = model.predict(Test_X)
test_Accuracy = accuracy_score(Test_Y, y_pred)*100

#Test Accuracy 
y_pred = model.predict(Train_X)
train_Accuracy = accuracy_score(Train_Y, y_pred)*100

crossvalMean = cross_val_score(model, X, y, cv=10).mean()

curTime = stop - start;
print(f"Training Time = {curTime:0.8f} Seconds")
print(f"Test Accuracy = {test_Accuracy}")
print(f"Training Accuracy = {train_Accuracy}")
print(f"Cross Validation Mean = {crossvalMean}")

Training Time = 0.39805280 Seconds
Test Accuracy = 70.41666666666667
Training Accuracy = 70.08928571428571
Cross Validation Mean = 0.69625
